## Libraries

In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.image import resize

In [2]:
# Loading Model
model = tf.keras.models.load_model('./Trained_model.h5')
classes = os.listdir('./Data/genres_original/')


In [3]:
# Preprocessing audio file before testing
def  load_and_preprocess_file(file_path, target_shape=(210,210)):
    data = []
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    chunk_duration = 4
    overlap_duration = 2
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate
    num_chunks = int(np.ceil((len(audio_data)-chunk_samples)/(chunk_samples - overlap_samples))+1)
    for i in range(num_chunks):
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples
        #extract the chunk
        chunk = audio_data[start:end]
        #Melspectrogram part
        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
        #Resizing the Matrix to the target shape
        mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
        #Append data to list
        data.append(mel_spectrogram)
    return np.array(data)

In [4]:
file_path = "./test_samples/test/08. No Rain - Blind Melon.mp3"
X_test = load_and_preprocess_file(file_path)

In [5]:
X_test.shape

(108, 210, 210, 1)

## Model Prediction 

In [6]:
def model_prediction(X_test):
    y_pred = model.predict(X_test)
    predicted_categories = np.argmax(y_pred, axis=1)
    unique_elements, counts = np.unique(predicted_categories, return_counts=True)
    max_count = np.max(counts)
    max_element = unique_elements[counts==max_count]
    return max_element[0], unique_elements, counts


In [7]:
# c_index = model_prediction(X_test)

In [8]:
c_index, u_elements, counts = model_prediction(X_test)

4/4 [==============================] - 37s 178ms/step


In [9]:
classes

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [10]:
total = sum(counts)
for value in counts:
    percentage = (value / total) * 100
    index = list(counts).index(value)
    print(f"Value: {[classes[i] for i in u_elements][index]}, Percentage: {percentage:.2f}%")

Value: blues, Percentage: 8.33%
Value: classical, Percentage: 2.78%
Value: country, Percentage: 32.41%
Value: classical, Percentage: 2.78%
Value: jazz, Percentage: 5.56%
Value: reggae, Percentage: 1.85%
Value: rock, Percentage: 46.30%


In [11]:
print("The predicted music genre is:", classes[c_index])

The predicted music genre is: rock
